In [2]:
#  pre-trained model densenet121
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
import time
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
from numpy import linalg as LA
import h5py
from skimage import io
import pymongo


# 圖片預處理
def preprocess_image(image):
    try:
        # load to gray image
        image = cv2.imread(image)
        ori = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(ori, cv2.COLOR_BGR2GRAY)

        # 用Sobel算子計算x，y方向上的梯度，之後在x方向上減去y方向上的梯度
        # 我們留下具有高水平梯度和低垂直梯度的圖像區域
        gradX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        gradY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)
        # subtract the y-gradient from the x-gradient
        gradient = cv2.subtract(gradX, gradY)
        gradient = cv2.convertScaleAbs(gradient)

        # blur and the image 去除圖像上的噪聲。
        blurred = cv2.blur(gradient, (9, 9))

        # threshold the image 對模糊圖像二值化
        (_, thresh) = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
        closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        # perform a series of erosions and dilations
        closed = cv2.erode(closed, None, iterations=4)  # 侵蝕
        closed = cv2.dilate(closed, None, iterations=58)  # 膨脹

        # find Contour
        (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        c = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

        # compute the rotated bounding box of the largest contour
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))

        # find image bounding box
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]

        x1 = min(Xs)
        y1 = min(Ys)
        x2 = max(Xs)
        y2 = max(Ys)

        if x1 < 0:
            x1 = 0
        if y1 < 0:
            y1 = 0

        hight = y2 - y1
        width = x2 - x1
        crop_img = image[y1:y1 + hight, x1:x1 + width]

        #         img_result = tf.image.resize_with_crop_or_pad(crop_img,590,590)
        img_result = tf.image.resize(crop_img, [224, 224])

        img_result = np.array(img_result)
        img_result = cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB)
        return img_result
    except IndexError:
        print("list out of range")
        return -1


# run model找出feature
def extract_feat(img_path):
    # input_shape = (224,224,3)
    # img = tf.keras.preprocessing.image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
    # img = keras.preprocessing.image.img_to_array(img)
    img = preprocess_image(img_path) # 圖像預處理
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    model_densenet = DenseNet121(weights ='imagenet',
                            input_shape = (224,224,3),
                            pooling = 'max', include_top = False)
    feat = model_densenet.predict(img)
    norm_feat = feat[0]/LA.norm(feat[0])
    return norm_feat

# Output imageUrl
def find_imgUrl(classNo, img_input):
    model_h5 = 'models/feature_%s.h5' % classNo   # feat.h5 檔案路徑
    h5f = h5py.File(model_h5, 'r')
    feats = h5f['dataset_1'][:]
    imgNames = h5f['dataset_2'][:]

    feat = extract_feat(img_input)
    scores = np.dot(feat, feats.T)
    rank_ID = np.argsort(scores)[::-1]

    imlist = []
    maxres = 3  # output 數量
    for i, score in enumerate(rank_ID[0:maxres]):
        imgid = str(imgNames[score], 'utf-8').split("_")[1] # 照片名稱 EX:"001_000000015" 取後段 imgId
        # print(imgid)
        imlist.append(imgid)
    myclient = pymongo.MongoClient('localhost', 27017)   # MongoDB (ip,27017)
    mydb = myclient.test  # dbName
    mycol = mydb.tmark_json  # collectionName
    results = mycol.find({})
    Urllist = []
    for result in results:
        if result["_id"] in imlist: # key
            imgurl = result["image"]
            Urllist.append(imgurl)
            # print(imgurl)
    return Urllist
# Output image
def find_img(classNo, img_input):
    model_h5 = 'models/feature_%s.h5' % classNo
    h5f = h5py.File(model_h5, 'r')
    feats = h5f['dataset_1'][:]
    imgNames = h5f['dataset_2'][:]

    feat = extract_feat(img_input)
    scores = np.dot(feat, feats.T)
    rank_ID = np.argsort(scores)[::-1]

    imlist = []
    maxres = 3  # output 數量
    for i, score in enumerate(rank_ID[0:maxres]):
        imgid = str(imgNames[score], 'utf-8').split("_")[1]
        # print(imgid)
        imlist.append(imgid)
    myclient = pymongo.MongoClient('localhost', 27017)
    mydb = myclient.test
    mycol = mydb.tmark_json
    results = mycol.find({})
    for result in results:
        if result["_id"] in imlist:
            image = io.imread(result["image"])
            io.imshow(image)
            io.show()

if __name__=="__main__":
    print(find_imgUrl('012','./database/089037461.jpg'))


000000015
000000025
000000022
['https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=/20160227/000/000/015/pic_000000015_20110105_1.jpg&formatName=jpeg&pathCodeId=282_pic', 'https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=/20160227/000/000/025/pic_000000025_20110105_1.jpg&formatName=jpeg&pathCodeId=282_pic', 'https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=/20160227/000/000/022/pic_000000022_20110105_1.jpg&formatName=jpeg&pathCodeId=282_pic']
None
